In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import struct
import socket
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import gc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the dataset
data_url = '/content/drive/MyDrive/Ngampus/train_test_network.csv'  # Update this with your local path
data = pd.read_csv(data_url)
df = pd.DataFrame(data)

# Preview the dataset
print(data.head())

          src_ip  src_port         dst_ip  dst_port proto service    duration  \
0   192.168.1.37      4444  192.168.1.193     49178   tcp       -  290.371539   
1  192.168.1.193     49180   192.168.1.37      8080   tcp       -    0.000102   
2  192.168.1.193     49180   192.168.1.37      8080   tcp       -    0.000148   
3  192.168.1.193     49180   192.168.1.37      8080   tcp       -    0.000113   
4  192.168.1.193     49180   192.168.1.37      8080   tcp       -    0.000130   

   src_bytes  dst_bytes conn_state  ...  http_response_body_len  \
0     101568       2592        OTH  ...                       0   
1          0          0        REJ  ...                       0   
2          0          0        REJ  ...                       0   
3          0          0        REJ  ...                       0   
4          0          0        REJ  ...                       0   

   http_status_code  http_user_agent  http_orig_mime_types  \
0                 0                -            

In [ ]:
# Basic preprocessing
data.fillna(0, inplace=True)  # Fill missing values
# Convert categorical columns to numerical (if any)
data = pd.get_dummies(data)

# Normalize the dataset
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data.drop('label', axis=1))  # Assuming 'label' is the target column

In [ ]:
from sklearn.model_selection import train_test_split

X = scaled_data
y = data['label'].values  # Assuming 'label' is your target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Create a tf.data.Dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.batch(100).prefetch(tf.data.AUTOTUNE)  # Adjust batch size as needed

In [ ]:
# Create a tf.data.Dataset for testing
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(100).prefetch(tf.data.AUTOTUNE)

In [ ]:
def build_smaller_transformer_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.MultiHeadAttention(num_heads=4, key_dim=32)(inputs, inputs)  # Fewer heads
    x = layers.BatchNormalization()(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)
    x = layers.Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(x)  # Fewer filters
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(32, activation='relu')(x)  # Smaller dense layer
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

model = build_smaller_transformer_model((X_train.shape[1], 1))  # Adjust input shape accordingly
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Reshape X_train and X_test for the transformer
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

model.fit(X_train_reshaped, y_train, epochs=10, batch_size=100, validation_data=(X_test_reshaped, y_test))

Epoch 1/10
 101/1689 [>.............................] - ETA: 2:45:01 - loss: 0.6807 - accuracy: 0.7618

In [ ]:
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f'Test Accuracy: {accuracy:.2f}')